# Glassdoor Reviews WebScraper

### Created by Shaik Nawazuddin

Importing all the necessary Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

Specifying User-Agent

Just google 'My user agent' and paste that as the value for key 'User-Agent'

In [ ]:
headers = {'User-Agent' : ''}

Creating a temporary list to append data of each page into it

In [ ]:
temp = list()

Specify the last page number

In [ ]:
start_page = 1
print('Please enter the last page number')
end_page = (int(input())) + 1

Example:

https://www.glassdoor.co.in/Reviews/Glassdoor-Reviews-E100431.htm

If the link looks like the above, kindly take only the 'https://www.glassdoor.co.in/Reviews/Glassdoor-Reviews-E100431' part as the first_half and I'd recommend not to change the second_half unless there is a specific filtering applied.

In [ ]:
for page in range(start_page, end_page):
    print('Page number is: ', page)

    try:
#Kindly specify the correct URL link in the right format without including the page number as shown in the example        
        first_half = 'https://www.glassdoor.co.in/Reviews/Glassdoor-Reviews-E100431'
        second_half = '.htm?filter.iso3Language=eng'     
        html = requests.get(first_half + '_P' + str(page) + second_half, verify = True, headers = headers)
        html.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print('HTTP Error: ', errh)
    except requests.exceptions.ConnectionError as errc:
        print('Error Connecting: ', errc)
        time.sleep(30)
    except requests.exceptions.Timeout as errt:
        print('Timeout Error: ', errt)
        time.sleep(30)
    except requests.exceptions.RequestException as err:
        print('Something else: ', err)
   
    bsobj = BeautifulSoup(html.text, 'html.parser')

    reviews = bsobj.find_all(class_='gdReview')
   
    for review in reviews:
        try:
            desc = review.find(class_='reviewLink').get_text()
        except:
            desc = None
        try:
            date_job_title = review.find(class_='authorJobTitle').get_text()
            date_job_title = date_job_title.split('-')
            review_date = date_job_title[0]
            reviewer_job_title = date_job_title[1]
        except:
            review_date = None
            reviewer_job_title = None
        try:
            location = review.find(class_='authorLocation').get_text()
        except:
            location = None
        try:
            rate = review.find(class_='ratingNumber').get_text()
        except:
            rate = None
        try:
            state = review.find(class_='pt-xsm').get_text()
        except:
            state_exp = None
        try:
            pros =review.select('span[data-test="pros"]')[0].text
            pros = pros.replace('\t', '')
            pros = pros.replace('\r', ' ')
            pros = pros.replace('\n', ' ')
        except:
            pros = None
        try:
            cons = review.select('span[data-test="cons"]')[0].text
            cons = cons.replace('\t', '')
            cons = cons.replace('\r', ' ')
            cons = cons.replace('\n', ' ')
        except:
            cons = None
       
        data = {
                'Title': desc,
                'Date': review_date,
                'Job Title': reviewer_job_title,
                'Location' : location,
                'Rating': rate,
                'State and Experience': state,
                'Pros' : pros,
                'Cons': cons
            }
        temp.append(data)

Loading the temp data into a DataFrame

In [ ]:
df_temp = pd.DataFrame(temp)
try:
    df_temp[['State of Reviewer', 'Experience']] = df_temp['State and Experience'].str.split(', ', expand = True)
except:
    df_temp['State of Reviewer'] = df_temp['State and Experience']
    df_temp['Experience'] = ''
df = df_temp[['Title', 'Date', 'Job Title', 'Location', 'Rating', 'State of Reviewer', 'Experience', 'Pros', 'Cons']]
df

Exporting the DataFrame to an excel file

In [ ]:
df.to_excel('Glassdoor Reviews.xlsx', index = False)